In [69]:
import numpy as np
import os
from  natsort import natsorted
import imageio
import re
import time
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input, GRU, Reshape
from keras.layers import Conv2D, MaxPooling2D, Bidirectional
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers.merge import add, concatenate
from keras import backend as K
from keras.models import load_model, Model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
import matplotlib.pyplot as plt
import itertools

In [7]:
NAME = 'ALPR_CNN'
data_dir = 'Data'
model_dir = 'Models'

In [39]:
def prepare_dataset(data_dir, folder_name):
    try:
        print('Loading numpy')
        X = np.load('X_{}.npy'.format(folder_name))
        y = np.load('y_{}.npy'.format(folder_name))

    except:
        print('Loading images')
        image_list = []
        labels = []
        pictures_dir = os.path.join(data_dir, folder_name)
        names = [ d for d in os.listdir( pictures_dir ) if d.endswith( '.jpg') ]
        names = natsorted(names)
        for image in names:
            image_list.append(imageio.imread(os.path.join(pictures_dir, image)))
            label = re.split('[._]', image)
            labels.append(label[0][2:])
            print(label[0][2:])
        X = np.stack(image_list, axis=0)
        y = np.array(labels)
        np.save(os.path.join(data_dir,'X_{}'.format(folder_name)),X)
        np.save(os.path.join(data_dir,'y_{}'.format(folder_name)),y)
    return X,y

In [47]:
X,y = prepare_dataset(data_dir, 'resized')

Loading numpy
Loading images
A9H707
A66666
A66666
A88888
A88888
A88888
A88888
A88888
A88888
A99999
AE4444
AG1111
AG6666
AKC700
AKC700
AL3333
AN7777
AQ7777
AVB515
B66666
B88888
B88888
B88888
B88888
C88888
D88888
DN1972
E88888
E88888
A00001
A629AT
A629AT
A999BG
A999BG
A01606
A01606
A2979J
A2979J
A2979J
A8400X
A8400Z
A8888S
A9988A
A9988A
A9988A
A9988A
A9988A
A33333
A33333
A77777
A77777
A84004
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
AB279N
AF5126
AGG888
AH0987
AH0987
AK8888
APA773
ASB110
ASF227
ASF227
AUL666
AUL666
AWB011
AWB011
AWT268
AWT268
B8888K
B8888K
B77777
B88888
BF8888
BM0902
BPM179
BPM179
C2B007
C2B007
C7S110
C7S110
C986E9
C4444L
C4444X
C4444X
C5942B
C11111
C44444
C66666
C77777
C80000
C83558
C88888
C88888
C88888
C90000
C90000
C99999
C99999
C99999
CT1259
D2D318
D5H666
D0008T
D0066T
D79F76
D1666T
D3000T
D5000T
D5550T
D6868T
D7778T
D8666T
D8686T
D8787T
D9000T
D9099T
D9888T
D9898T
D9999Q
D9999Q
D12345
D12345
D33333
D33333
D55555
D55555
D55555
D66666
D77777
D8008

C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88888
C88889
C88889
C99999
CEE732
CEE732
CEJ169
CGU999
CL7N05
CL7N05
CRV738
CRV738
CS8417
CS8417
CSB123
CSB123
CSB123
CYN831
D8K710
D8K710
D9J929
D9J929
D29K27
D00818
D00818
D9770K
D9770K
D11111
D88888
D88888
D88888
D88888
DA2F12
DA2F12
DAA000
E66666
E66666
E66666
E99999
EL9999
EY6166
F8L557
F5008Q
F5008Q
F20008
F20008
F20008
F20008
F22222
F33333
F77777
F77777
F77777
F77777
F77777
F77777
F77777
F82222
F82222
F88888
F99999
F99999
FM9000
FP0628
FP9991
G0C001
G00001
G520DX
G520DX
G66666
G88888
GC8733
GC9999
GC9999
GC9999
GC9999
GC9999
GC9999
GMA888
GMA888
GMK888
GZC333
GZC333
HA8888
HA8888
HG8888
J98X79
J5912B
J5942B
J5942B
J5942B
JEK562
K12029H
KF5907
KR8672
0A3522
0A8610
0AB610
000000
000000
000000
000000
000000
000000
000000
000098
099999
A0WL02
A0WL02
A2CN06
A31F35
A088TZ
A250D1
A444F3
A506T1
A575FE
A575FE
A833HR
A833HR
A888U8
A888U8
A906L

FS7777
G22222
G33333
GAD567
GBF345
GD0567
GD5678
GDA876
GDD876
GDU789
GFK789
A00000
A00000
A00000
A00000
A000000
A00000
A00001
A00001
A00001
A00001
A00001
A00001
A00001
A00003
A00003
A00817
A00817
A10000
A10000
A10000
A10000
A10000
A11111
A12378
A12378
A22222
A31541
A31541
A33333
A61169
A61169
A66666
A66666
A80293
A80987
A80987
A81255
A81576
A88050
A88050
A88050
A88050
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A88888
A89106
A99999
A99999
A99999
A333333
AA8300
AC9177
AF0008
AF0008
AF0236
AF0236
AG6019
AG6104
AG6104
AG6104
AG6118
AG6123
AG6151
AG6182
AK9999
AK9999
AK9999
AK9999
AV3200
AX2089
AX2089
AX8888
CV8888
CX8888
CX8888
CX8888
CX8888
CX8888
E76543
E88888
EH0000
EP0000
EP0000
EX9201
EX9201
EX9201
F00544
F59140
F88888
F88888
FA1326
FA1326
FH0000
FH0000
FH00

BB8888
BB8888
BF395V
BF395V
BF395V
BF395V
BJG923
BR0000
BWP000
BWP000
C88888
C88888
CA220F
CZ9874
CZV987
CZV987
CZV987
CZV987
CZV987
CZV987
D3Q520
D3Q520
D3Q520
D3Q520
D00008
D44444
D44444
D88888
D88888
D88888
D88888
D88888
D88888
DC8000
DC8000
DD335G
DF7777
DJB255
DJB255
DN5092
DQD862
DS509A
DV0001
DV0001
E2J3Y2
E2LJ88
E3KK86
E23B79
E28J78
E28J78
E31R79
E85P13
E85P13
E128ML
E187L8
E187L8
E520Q3
E520Q3
E772Z5
E772Z5
E888K8
E953B8
E953B8
E991N6
E1315H
E3333U
E8200Z
E8886Z
E8888Z
E45678
E77777
E99271
E99999
EA13X7
EA13X7
EC35A6
EC53A6
EC62N8
EC62N8
EC62N8
EC62N8
EC88H8
EC88H9
ED0000
ED0000
ED68P6
EE00C0
EE22C7
EE22V5
EE52V5
EF92E9
EG00P1
EG00P1
EG00P1
EG26J6
EG26J6
EG26J6
EM55Q7
EM86B8
EM87B8
EM87B8
EN23K1
EN78R7
EN78R7
EN78R7
ENV888
EQ922Y
ES70B3
ESB336
ESB336
ESB748
ESB748
ESB748
ESB748
ESB748
ET07Z8
EV4605
EV4605
F23B79
FLU438
G10086
G10086
G88888
G88888
G88888
GA0000
GA0000
H88888
HDQ306
HS4865
J051B5
J6666B
J6666B
JAA458
JAA458
JEU433
JJF999
K92299
K92299
L88888
LX9777
LZ5757
M53D88

In [53]:
X.shape

(3921, 224, 224, 3)

In [58]:
np.random.seed(42)
indices = np.arange(X.shape[0])
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

In [60]:
z = 3500
X_train = X[:z]
X_test = X[z:]
y_train = y[:z]
y_test = y[z:]

(3500, 224, 224, 3)

## Keras Model

In [50]:
img_w = 224
# Input Parameters
img_h = 224
# Network parameters
conv_filters = 64
kernel_size = (3, 3)
pool_size = 2
time_dense_size = 32
rnn_size = 64
minibatch_size = 128
unique_tokens = 37

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_w, img_h)
else:
    input_shape = (img_w, img_h, 1)


In [74]:
act = 'relu'
input_data = Input(name='the_input', shape=input_shape, dtype='float32')
inner = Conv2D(conv_filters, kernel_size, padding='same',
               activation=act, kernel_initializer='he_normal',
               name='conv1')(input_data)
inner = MaxPooling2D(pool_size=(pool_size, pool_size), name='max1')(inner)
inner = Conv2D(conv_filters, kernel_size, padding='same',
               activation=act, kernel_initializer='he_normal',
               name='conv2')(inner)
inner = MaxPooling2D(pool_size=(pool_size, pool_size), name='max2')(inner)

conv_to_rnn_dims = (img_w // (pool_size ** 2), (img_h // (pool_size ** 2)) * conv_filters)
inner = Reshape(target_shape=conv_to_rnn_dims, name='reshape')(inner)

# cuts down input size going into RNN:
inner = Dense(time_dense_size, activation=act, name='dense1')(inner)

# Two layers of bidirectional GRUs
# GRU seems to work as well, if not better than LSTM:
gru_1 = GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal', name='gru1')(inner)
gru_1b = GRU(rnn_size, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b')(inner)
gru1_merged = add([gru_1, gru_1b])
gru_2 = GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal', name='gru2')(gru1_merged)
gru_2b = GRU(rnn_size, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru2_b')(gru1_merged)

# transforms RNN output to character activations:
inner = Dense(unique_tokens, kernel_initializer='he_normal',
              name='dense2')(concatenate([gru_2, gru_2b]))
y_pred = Activation('softmax', name='softmax')(inner)
Model(inputs=input_data, outputs=y_pred).summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 224, 224, 1)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 224, 224, 64) 640         the_input[0][0]                  
__________________________________________________________________________________________________
max1 (MaxPooling2D)             (None, 112, 112, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 112, 112, 64) 36928       max1[0][0]                       
__________________________________________________________________________________________________
max2 (MaxP

In [80]:
y_train[1]

'C99999'